In [ ]:
# ===============================
# Connect Four Simulation Notebook
# ===============================

# --- Imports ---
import sys
import os
import numpy as np
from itertools import combinations

# --- Make sure SRC folder is on path ---
if '../src' in sys.path:
    sys.path.remove('../src')
sys.path.append('../src')

# --- Import functions from SRC ---
from board import create_board, make_move, check_win
from player import random_player, heuristic_player, intelligent_player
from simulation import simulate_game  # your simulate_game function

# --- Parameters ---
BASE_SIM = 100  # default number of simulations per pairing

# --- Define players ---
players = {
    "4 Years": random_player,
    "7 Years": heuristic_player,
    "9 Years": lambda b, p: intelligent_player(b, p, depth=1),
    "11 Years": lambda b, p: intelligent_player(b, p, depth=2)
}

# --- Automatic simulation count per pairing (100–500) ---
def determine_sim_count(balance_estimate):
    """Return number of simulations based on estimated balance (0=unbalanced,1=balanced)"""
    return int(np.clip(BASE_SIM * (1 + balance_estimate * 2), 100, 500))

# --- Estimated balance between players ---
balance_estimates = {
    ("4 Years","7 Years"): 0.1,
    ("4 Years","9 Years"): 0.3,
    ("4 Years","11 Years"): 0.05,
    ("7 Years","9 Years"): 0.5,
    ("7 Years","11 Years"): 0.7,
    ("9 Years","11 Years"): 0.8
}

# --- Run simulations and print results ---
for p1_name, p2_name in combinations(players.keys(), 2):

    # Number of simulations for this pairing
    sim_count = determine_sim_count(balance_estimates.get((p1_name, p2_name), 0.3))

    # Initialize outcomes
    outcomes = {p1_name:0, p2_name:0, "Draw":0}

    # Run simulations
    for _ in range(sim_count):
        winner = simulate_game(players[p1_name], players[p2_name])
        if winner == 1:
            outcomes[p1_name] += 1
        elif winner == 2:
            outcomes[p2_name] += 1
        else:
            outcomes["Draw"] += 1

    # Calculate probabilities
    prob_p1 = outcomes[p1_name] / sim_count
    prob_p2 = outcomes[p2_name] / sim_count
    prob_draw = outcomes["Draw"] / sim_count

    # Standard errors
    se_p1 = np.sqrt(prob_p1 * (1 - prob_p1) / sim_count)
    se_p2 = np.sqrt(prob_p2 * (1 - prob_p2) / sim_count)
    se_draw = np.sqrt(prob_draw * (1 - prob_draw) / sim_count)

    # Betting odds (simple)
    quote_p1 = 1 / prob_p1 if prob_p1 > 0 else float('inf')
    quote_p2 = 1 / prob_p2 if prob_p2 > 0 else float('inf')
    quote_draw = 1 / prob_draw if prob_draw > 0 else float('inf')

    # --- Print results ---
    print(f"\n{p1_name} vs {p2_name} ({sim_count} games)")
    print(f"{p1_name}: {prob_p1*100:.1f}% ± {se_p1*100:.1f}% → Quote ≈ {quote_p1:.2f}")
    print(f"{p2_name}: {prob_p2*100:.1f}% ± {se_p2*100:.1f}% → Quote ≈ {quote_p2:.2f}")
    print(f"Draw: {prob_draw*100:.1f}% ± {se_draw*100:.1f}% → Quote ≈ {quote_draw:.2f}")


NameError: name 'random_player' is not defined